In [ ]:
import numpy as np
import scrublet as scr
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
from collections import Counter
import anndata as ann
import bbknn
import cellrank as cr
import scanpy as sc
import scvelo as scv
import loompy

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

In [ ]:
adata = sc.read_h5ad("./h5ad/DNT_velocity_input.h5ad")

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=30, min_shared_cells=30, log=True)
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30, random_state=0)
scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

In [ ]:
scv.tl.recover_dynamics(adata, n_jobs=80)

In [ ]:
scv.tl.velocity(adata, mode="dynamical",min_r2=None)

In [ ]:
vk = cr.kernels.VelocityKernel(adata)

In [ ]:
vk.compute_transition_matrix()

In [ ]:
ck = cr.kernels.ConnectivityKernel(adata)
ck.compute_transition_matrix()

combined_kernel = 0.8 * vk + 0.2 * ck

In [ ]:
print(combined_kernel)

In [ ]:
vk.plot_projection()

In [ ]:
scv.settings.set_figure_params('cellrank', dpi=300)
vk.plot_projection(basis='umap',color='DNT_C',
                                 legend_loc='right margin',legend_fontsize=5,smooth=0.8, min_mass=2.5,
                                 save="DNT_cellrank_stream.pdf", figsize=(5,5))